# Introduction to Machine Learning, UZH 2018, Group Project
### Group 2: Barbara Capl, Mathias Lüthi, Pamela Matias, Stefanie Rentsch
##    
###   
# 3. Classification/ Prediction 
# B. with RandomForestClassifier (RF)
###   
In this section we use the selected features matrices, pre-selected with the RandomForestClassifier in chapter 2 or with PCA in chapter 2i and the response vector that we prepared in chapter 1.  

#### We use two different versions (created in chapter 1, features-selected in chapter 2):
Version 1: Feature Matrix consists only of the Ratios                                                                        
Version 2: Feature Matrix consists of Ratios + dummy variables for seasonality + other market data
####  We will do Classification and Prediction with RandomForestClassifier


In [112]:
# hide unnecessary warnings ("depreciation" of packages etc.)
import warnings
warnings.filterwarnings('ignore')

# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier

###   
###   
## 3.0. SETTINGS

### (1) Choose the Dataset Version you want

##### Whole Feature Matrix (Features not pre-selected)
VERSION = 1; Feature Matrix with only ratios                                  
VERSION = 2;  Feature Matrix with ratios + saisonality + other market data

##### Reduced Feature Matrix (Features pre-selected)
VERSION = 1.1; Reduced Feature Matrix with only ratios                                  
VERSION = 2.1;  Reduced Feature Matrix with ratios + saisonality + other market data



In [113]:
### Chose which dataset version you want the selection of features and the prediction to be based on 
VERSION = 1
"""
INSERT NUMBER 1, 2, 1.1 or 2.1
"""



# Defining sel_state variable for easier printing out    
if VERSION == 1:
    sel_version = 'Based on whole original Dataset with only the Ratios Dataset as predicive Features.'
elif VERSION == 2:
    sel_version = 'Based on whole original Dataset with Ratios + Seasonality + other Market Data as predictive Features.'
elif VERSION == 1.1:
    sel_version = 'Based on reduced Dataset with only the Ratios Dataset as predicive Features.'
elif VERSION == 2.1:
    sel_version = 'Based on reduced Dataset with Ratios + Seasonality + other Market Data as predictive Features.'
else: raise ValueError('VERSION must be either 1, 2, 1.1 or 2.1')

### (2) If you chose VERSION 1.1. or VERSION 2.1:  (Reduced Feature Matrix)                                                                  
### => Choose with which method you want to have the features been pre-selected /reduced

##### You have the choice between:
mySELECTION  = RF ; Features pre-selected with Random Forest Classifier                                                           
mySELECTION = PCA; Features pre-selected with Principal Component Analysis (PCA)                                         

##### By Default;
If VERSION 1 or VERSION 2 was chosen above: SELECTION = none by Default; no features pre-selected. You don't need to define variable mySELECTION.

In [114]:
### Choose whether you want the datasets with features selected with RF or PCA or the original file
mySELECTION = 'RF'
"""
INSERT WISHED METHOD 'RF', 'PCA'
"""



# This is the control loop. If something has been chosen wrong, it returns an error with explanation.
if VERSION == 1 or VERSION == 2:
    SELECTION = 'none'
elif VERSION == 1.1 or VERSION == 2.1:
    SELECTION = mySELECTION
    if mySELECTION is not 'RF' and mySELECTION is not 'PCA':
        raise ValueError('Because VERSION '+str(VERSION)+' is chosen, mySELECTION must be set as either RF or PCA.')
else: raise ValueError('VERSION must be either 1, 2, 1.1 or 2.1. mySELECTION must be chosen as either RF or PCA.')

# Defining of sel_feat (Selected Feature Selection Method) variable and briefing for later.   
if SELECTION == 'RF':
    sel_feat = 'Random Forest (RF)'
    briefing = ('You chose dataset VERSION '+str(VERSION)+' and SELECTION method '+str(SELECTION)+'.'+'\n'+'Features therefore pre-selected with '+str(sel_feat)+'.')
elif SELECTION == 'PCA':
    sel_feat = 'Principal Component Analysis (PCA)'
    briefing = ('You chose dataset VERSION '+str(VERSION)+' and SELECTION method '+str(SELECTION)+'.'+'\n'+'Features therefore pre-selected with '+str(sel_feat)+'.')
elif SELECTION == 'none':
    sel_feat = 'No Feature Selection Method available.'
    briefing = ('You chose VERSION '+str(VERSION)+'. This Version has no Feature Selection Method because Feature Matrix is whole, not reduced.'+'\n'+'SELECTION is therefore "none" by Default.')
else: raise ValueError('mySELECTION must be chosen as either RF or PCA')
print('You chose SELECTION method '+str(sel_feat)+'.')
#print(sel_feat)

You chose SELECTION method No Feature Selection Method available..


### (3) SUMMARY OF SETTINGS

In [115]:
print(briefing, '\n')
print('VERSION '+str(VERSION)+' is '+str(sel_version),'\n')
print('You are now done with the Settings. You can run the whole Code now by Default.')

You chose VERSION 1. This Version has no Feature Selection Method because Feature Matrix is whole, not reduced.
SELECTION is therefore "none" by Default. 

VERSION 1 is Based on whole original Dataset with only the Ratios Dataset as predicive Features. 

You are now done with the Settings. You can run the whole Code now by Default.


###  
###   
## 3.1. Preparation

Note: No Standardization needed for RF.

### 3.1.1. Import the Response Vector and the Feature Matrix

In [116]:
####################### NEW COMMENT
# In version 1 und 2; ganze Feature matrix und ganzen Response vector rein? ODER gesplittet?
# dann müsste in DataPrep split gemacht werden für datenset ohne feature pre-selection
# abgespeichert in generated_splits ohne Unterordner
# hier eingelesen als X_train und y_train und X_test und y_test
#######################


### import Data (already splitted to train/test-data and selected features-> bc_randomforest_feature_selection)
if VERSION == 1: 
# features not pre-selected, only ratios
    X = pd.read_csv('Data/generated_datasets/features_ratios_1.csv', sep=',', header=0)
    y = pd.read_csv('Data/generated_datasets/response_1.csv', sep=',', header=0)
elif VERSION == 2: 
# features not pre-selected, ratios + seasonality + market data
    X = pd.read_csv('Data/generated_datasets/features_additional_1.csv', sep=',', header=0)
    y = pd.read_csv('Data/generated_datasets/response_1.csv', sep=',', header=0)
elif VERSION == 1.1: 
# features pre-selected, only ratios
    if SELECTION == 'RF':
        X_train_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/X1_train_f.csv', sep=',', header=0)
        X_test_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/X1_test_f.csv', sep=',', header=0)
        y_train_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/y1_train_f.csv', sep=',', header=0)
        y_test_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/y1_test_f.csv', sep=',', header=0)
    elif SELECTION == 'PCA':
        X_train_s = pd.read_csv('Data/generated_splits/features_selected_pca/X1_train_p.csv', sep=',', header=0)
        X_test_s = pd.read_csv('Data/generated_splits/features_selected_pca/X1_test_p.csv', sep=',', header=0)
        y_train_s = pd.read_csv('Data/generated_splits/features_selected_pca/y1_train_p.csv', sep=',', header=0)
        y_test_s = pd.read_csv('Data/generated_splits/features_selected_pca/y1_test_p.csv', sep=',', header=0)
elif VERSION == 2.1: 
# features pre-selected, ratios + seasonality + market data
    if SELECTION == 'RF':
        X_train_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/X2_train_f.csv', sep=',', header=0)
        X_test_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/X2_test_f.csv', sep=',', header=0)
        y_train_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/y2_train_f.csv', sep=',', header=0)
        y_test_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/y2_test_f.csv', sep=',', header=0)
    elif SELECTION == 'PCA':
        X_train_s = pd.read_csv('Data/generated_splits/features_selected_pca/X2_train_p.csv', sep=',', header=0)
        X_test_s = pd.read_csv('Data/generated_splits/features_selected_pca/X2_test_p.csv', sep=',', header=0)
        y_train_s = pd.read_csv('Data/generated_splits/features_selected_pca/y2_train_p.csv', sep=',', header=0)
        y_test_s = pd.read_csv('Data/generated_splits/features_selected_pca/y2_test_p.csv', sep=',', header=0)
else: raise ValueError('VERSION value must be either 1, 2, 1.1 or 2.1, mySELECTION must be chosen as either RF or PCA.')

#### Train-Test-split for whole original files. Automatically executed only if VERSION = 1 or 2.

In [117]:
####################### NEW COMMENT
# Split could also be already done in Datapreparation file because there we have more space etc.^ maybe
# but it can also be made here doesnt matterrr
#######################


# For VERSION == 1 or 2 -> train-test-split for the importet sets must be done
if VERSION == 1 or VERSION == 2:
    # import package imputer
    from sklearn.preprocessing import Imputer
    # # Train/test split, into 20% test size and 80% train size because it is a relatively small dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Use a median fill for train
    imp = Imputer(missing_values=np.nan, strategy = 'median' , axis=0)
    imputed_dataset = pd.DataFrame(imp.fit_transform(X_train))
    imputed_dataset.columns = X_train.columns
    imputed_dataset.index = X_train.index
    X_train = imputed_dataset

    # Use a median fill for the test set
    imputed_dataset = pd.DataFrame(imp.fit_transform(X_test))
    imputed_dataset.columns = X_test.columns
    imputed_dataset.index = X_test.index
    X_test = imputed_dataset
    
    # Extract the feature labels
    feature_labels = list(X)
    print('Type of feature_labels = ' + str(type(feature_labels)), '\n')
    
else: print('No Train/Test split needed for chosen VERSION '+str(VERSION)+
            '. Loaded Datasets were already pre-splitted and imputed in'+'\n'+'Feature Selection (Chapter 2).')

Type of feature_labels = <class 'list'> 



### 3.1.2. Print out Shape and Form of Feature Matrix and Response Vector

### Train Set

In [118]:
# print status
print('Features Selected with ' + str(sel_feat)+'.')
print('Version ' + str(VERSION) + '; ' + str(sel_version), '\n')

# print properties and head
if VERSION == 1 or VERSION == 2:
    print('Shape (rows, columns) of Feature Matrix X (Train) ' + '= ' + str(X_train.shape)+'\n')
    print('Feature Matrix X (Train) with no Feature pre-Selection:')
    display(X_train[0:3])
    print("")
    print('Response Vector y (Train) after no Feature pre-Selection:')
    display(y_train[0:3])
    print("")
else:
    print('Shape (rows, columns) of Feature Matrix X (Train) ' + '= ' + str(X_train_s.shape), '\n')
    print('Feature Matrix X (Train) with Selected Features:'+'\n')
    display(X_train_s[0:3])
    print("")
    print('Response Vector y (Train) after Feature Selection:')
    display(y_train_s[0:3])
    print("")

Features Selected with No Feature Selection Method available..
Version 1; Based on whole original Dataset with only the Ratios Dataset as predicive Features. 

Shape (rows, columns) of Feature Matrix X (Train) = (2836, 71)

Feature Matrix X (Train) with no Feature pre-Selection:


,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,pcf,dpr,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward
1530,20.773,0.547,10.644,15.468,15.633,18.484,18.484,3.703,9.957,0.623,...,1.4240,0.136,0.058,0.000,0.048,1.811,1.181,0.0354,13.678,5.963
1397,25.089,0.461,9.246,17.203,17.296,39.232,39.232,0.689,8.509,2.175,...,4.9910,0.031,0.000,0.000,0.047,2.189,0.668,0.0398,1.521,5.702
2238,12.337,0.911,12.474,9.135,9.209,13.062,13.062,2.034,1.993,0.337,...,5.8105,0.000,0.024,0.291,0.038,1.076,3.048,0.0268,0.380,2.248



Response Vector y (Train) after no Feature pre-Selection:


,0
1530,0
1397,1
2238,0


### Test Set

In [119]:
# print status
print('Features Selected with ' + str(sel_feat))
print('Version ' + str(VERSION) + '; ' + str(sel_version),'\n')

# print properties and head
if VERSION == 1 or VERSION == 2:
    print('Shape (rows, columns) of Feature Matrix X (Test) ' + '= ' + str(X_test.shape)+'\n')
    print('Feature Matrix X (Test) with no Feature pre-Selection:')
    display(X_test[0:3])
    print("")
    print('Response Vector y (Test) after no Feature pre-Selection:')
    display(y_test[0:3])
    print("")
else:
    print('Shape (rows, columns) of Feature Matrix X (Test) ' + '= ' + str(X_test_s.shape)+'\n')
    print('Feature Matrix X (Test) with Selected Features:')
    display(X_test_s[0:3])
    print("")
    print('Response Vector y (Test) after Feature Selection:')
    display(y_test_s[0:3])

Features Selected with No Feature Selection Method available.
Version 1; Based on whole original Dataset with only the Ratios Dataset as predicive Features. 

Shape (rows, columns) of Feature Matrix X (Test) = (710, 71)

Feature Matrix X (Test) with no Feature pre-Selection:


,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,pcf,dpr,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward
817,20.983,0.254,8.585,16.127,16.224,16.224,16.224,3.318,10.484,0.287,...,6.768,0.031,0.006,0.000,0.084,4.189,1.022,0.0187,1.414,1.330
2592,21.663,0.235,13.619,16.311,16.489,18.555,18.555,2.379,7.709,0.199,...,7.169,0.000,0.000,0.195,0.039,4.281,1.147,0.0122,1.737,1.572
1475,16.327,0.513,6.716,9.520,9.520,15.681,15.550,3.293,8.490,0.792,...,1.286,0.162,0.054,0.000,0.077,2.198,1.022,0.0343,-0.973,6.149



Response Vector y (Test) after no Feature pre-Selection:


,0
817,1
2592,0
1475,1


###   
###   
## 3.2. Random Forest Classification and Prediction

### 3.2.1. FItting Forest for single random_state and Prediction

In [120]:
# Define Variables
if VERSION == 1 or VERSION == 2: 
    feature_train = X_train
    feature_test = X_test
    response_train = y_train
    response_test = y_test
else:
    feature_train = X_train_s
    feature_test = X_test_s
    response_train = y_train_s
    response_test = y_test_s
    
# Fit forest
# the '_o' stands for 'prediction with forest with only ONE random state'
forest_o = RandomForestClassifier(random_state = 0)
forest_o.max_depth = 6
forest_o.fit(feature_train, response_train)

# Test prediction of y_test with the X_test feature matrix: gives the prediction vector
prediction_o = forest_o.predict(feature_test)

# Extract labels of features
feature_labels = feature_train.columns[:]
#display(feature_labels)

# Generate feature iportances
feature_importances_o = forest_o.feature_importances_

# sort features in line with their importance for the prediction (therefore ::-1)
indices_o = np.argsort(feature_importances_o)[::-1]

# Calculate percentage of of ones in the test response vector
print('Ratio of Ones (Test) =  ' + str(round(response_test['0'].sum() / response_test.size, 8)*100)+'%')
# Just to be sure the ones are distributed ca.the same in test and train response vector, check this:
print('Ratio of Ones (Training) =  ' + str(round(response_train['0'].sum() / response_train.size, 8)*100)+'%')
# Calculate precentage of ones predicted with the model
print('Score (Prediction) =  ' + str(round(prediction_o.sum() / prediction_o.size, 8)*100)+'%'+'\n')

# Calculate the score surplus above the test-set response vector score
print('EVALUATION: Our Model predicts '+str(round((prediction_o.sum()/prediction_o.size) 
                                       - (response_test['0'].sum()/response_test['0'].size),4)*100)+
      ' percentage points better than always guessing "UP" for the Test Set.')

Ratio of Ones (Test) =  57.042254%
Ratio of Ones (Training) =  58.074753%
Score (Prediction) =  81.267606%

EVALUATION: Our Model predicts 24.23 percentage points better than always guessing "UP" for the Test Set.


### 3.2.2. Fitting Forest and Prediction for multiple random_states

### Fit and predict with loop

In [121]:
# Define Variables
if VERSION == 1 or VERSION == 2: 
    feature_train = X_train
    feature_test = X_test
    response_train = y_train
    response_test = y_test
else:
    feature_train = X_train_s
    feature_test = X_test_s
    response_train = y_train_s
    response_test = y_test_s

# rs = first random state we choose
# i = number of random states we want to have tried out and the number of predictions we want to make
rs = 0
i = 10

df_prediction = pd.DataFrame()

# Loop until random_state = rs = 10
while rs < i:
    forest_m = RandomForestClassifier(random_state = rs)
    forest_m.max_depth = 6
    forest_m.fit(feature_train, response_train)
    df_prediction[rs] = forest_m.predict(feature_test)
    rs += 1

# Generate feature iportances
feature_importances_m = forest_m.feature_importances_

# sort features in line with their importance for the prediction (therefore ::-1)
indices_m = np.argsort(feature_importances_m)[::-1]
print('Predictions for random states 0 to 10 are: '+'\n')
display(df_prediction.head())
print("")
print('At the end of the loop, rs = ' + str(rs))

del rs, i

Predictions for random states 0 to 10 are: 



,0,1,2,3,4,5,6,7,8,9
0,1,1,1,1,1,1,1,1,1,1
1,0,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,0
3,1,1,1,0,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1



At the end of the loop, rs = 10


### Reduce all predictions into one representative one, then make a new column with rounded bools

In [122]:
# Make new column where sum of row is set relative to the numer of columns
# with this we kno, how many percent there was an "UP" predicted of how many times it could have been predicted
# the _m after the name stands for "multiple"

prediction_m = pd.DataFrame()
prediction_m['sum_prediction/col'] = df_prediction.sum(axis=1)/len(df_prediction.columns)
#display(res_prediction.head(5))

prediction_m['resulting_prediction'] = prediction_m['sum_prediction/col'].round()
display(prediction_m.head())

,sum_prediction/col,resulting_prediction
0,1.0,1.0
1,0.9,1.0
2,0.9,1.0
3,0.9,1.0
4,1.0,1.0


### Print Results of Prediction for multiple random states

In [123]:
# Calculate percentage of of ones in the test response vector
print('Ratio of Ones (Test) =  ' + str(round(response_test['0'].sum() / response_test['0'].size, 2)*100)+'%')
# Just to be sure the ones are distributed ca.the same in test and train response vector, check this:
print('Ratio of Ones (Training) =  ' + str(round(response_train['0'].sum() / response_train['0'].size, 6)*100)+'%')
# Calculate precentage of ones predicted with the model
print('Score (Prediction) =  ' 
      + str(round(prediction_m.resulting_prediction.sum() / prediction_m.resulting_prediction.size, 2)*100)+'%'+'\n')

# Calculate the score surplus above the test-set response vector score
print('EVALUATION: Our Model predicts '
      +str(round((prediction_m.resulting_prediction.sum()/prediction_m.resulting_prediction.size) 
                                       - (response_test['0'].sum()/response_test['0'].size),3)*100)+
      ' percentage points better than always guessing "UP" for the Test Set.')

Ratio of Ones (Test) =  56.99999999999999%
Ratio of Ones (Training) =  58.0748%
Score (Prediction) =  84.0%

EVALUATION: Our Model predicts 26.5 percentage points better than always guessing "UP" for the Test Set.


###   
###   
## 3.3. Cross-Validation

### 3.3.1. kFold with stratifying

In [124]:
# Import necessary function
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Create k-Fold CV and LogReg object
wished_splits = 5
kFold = StratifiedKFold(wished_splits, random_state=0)

# Run CV and print results
scores_o = cross_val_score(forest_o, feature_train, response_train, cv=kFold)
scores_m = cross_val_score(forest_m, feature_train, response_train, cv=kFold)

# Print Results
print('Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(scores_o)
print('CV accuracy on train set: {0: .3f} +/- {1: .3f}'.format(np.mean(scores_o), np.std(scores_o)),'\n')
print('Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print(scores_m)
print('CV accuracy on train set: {0: .3f} +/- {1: .3f}'.format(np.mean(scores_m), np.std(scores_m)))

Forest fitted and response Vector predicted with one random_state in 3.2.1.:
[0.5528169  0.61267606 0.57495591 0.5696649  0.60070671]
CV accuracy on train set:  0.582 +/-  0.022 

Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
[0.56338028 0.58626761 0.53968254 0.56790123 0.55123675]
CV accuracy on train set:  0.562 +/-  0.016


### 3.3.2. kFold with shuffle

In [125]:
# Import necessary function
from sklearn.model_selection import KFold

# Create k-Fold CV
wished_splits = 5
kFold = KFold(wished_splits, shuffle=True, random_state=0)

# Run CV
scores_o = cross_val_score(forest_o, feature_train, response_train, cv=kFold)
scores_m = cross_val_score(forest_m, feature_train, response_train, cv=kFold)

# Print Results
print('Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(scores_o)
print('CV accuracy on train set: {0: .3f} +/- {1: .3f}'.format(np.mean(scores_o), np.std(scores_o)),'\n')
print('Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print(scores_m)
print('CV accuracy on train set: {0: .3f} +/- {1: .3f}'.format(np.mean(scores_m), np.std(scores_m)))

Forest fitted and response Vector predicted with one random_state in 3.2.1.:
[0.61443662 0.58730159 0.55555556 0.55731922 0.56261023]
CV accuracy on train set:  0.575 +/-  0.023 

Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
[0.57394366 0.61022928 0.59611993 0.55555556 0.57671958]
CV accuracy on train set:  0.583 +/-  0.019


### 3.3.3. ROC-Area under the ROC Curve

In [126]:
# Run CV
scores_o = cross_val_score(forest_o, feature_train, response_train, cv=kFold, scoring='roc_auc')
scores_m = cross_val_score(forest_m, feature_train, response_train, cv=kFold, scoring='roc_auc')

# Print Reuslts
print('Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print('CV AUC on train set: {0: .3f} +/- {1: .3f}'.format(np.mean(scores_o), np.std(scores_o)),'\n')
print('Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print('CV AUC on train set: {0: .3f} +/- {1: .3f}'.format(np.mean(scores_m), np.std(scores_m)))

Forest fitted and response Vector predicted with one random_state in 3.2.1.:
CV AUC on train set:  0.558 +/-  0.020 

Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
CV AUC on train set:  0.567 +/-  0.011


### 3.3.4. Cross Validation Function (fit time, score time, accuracy, recall, roc_auc)

In [127]:
# Import necessary function
from sklearn.model_selection import cross_validate

# Definitions
measures = ['accuracy', 'recall', 'roc_auc']
number_of_jobs = 2

# Calculate scores
scores_o = cross_validate(forest_o, feature_train, response_train, cv=kFold, scoring=measures, n_jobs=number_of_jobs)
scores_o
scores_m = cross_validate(forest_m, feature_train, response_train, cv=kFold, scoring=measures, n_jobs=number_of_jobs)
scores_m

{'fit_time': array([0.09075952, 0.09116626, 0.08450937, 0.09208989, 0.05903864]),
 'score_time': array([0.00902152, 0.00850129, 0.00657153, 0.01059937, 0.00532484]),
 'test_accuracy': array([0.57394366, 0.61022928, 0.59611993, 0.55555556, 0.57671958]),
 'test_recall': array([0.82789318, 0.82544379, 0.8030303 , 0.81114551, 0.88714734]),
 'test_roc_auc': array([0.57271957, 0.57575386, 0.55630354, 0.55023347, 0.5787744 ]),
 'train_accuracy': array([0.69312169, 0.70074923, 0.69898634, 0.69766417, 0.69678272]),
 'train_recall': array([0.91679389, 0.9052712 , 0.92634776, 0.92749245, 0.91415663]),
 'train_roc_auc': array([0.78685278, 0.77111862, 0.78397475, 0.77698253, 0.78009087])}

In [128]:
# Print results
print('Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print('Train set accuracy (CV=5):    ', scores_o['train_accuracy'].mean())
print('Validation set scores (CV=5): ', scores_o['test_accuracy'].mean())
print('Test set accuracy:            ', forest_o.fit(feature_test, response_test).score(feature_test, response_test),'\n')

print('Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print('Train set accuracy (CV=5):    ', scores_m['train_accuracy'].mean())
print('Validation set scores (CV=5): ', scores_m['test_accuracy'].mean())
print('Test set accuracy:            ', forest_m.fit(feature_test, response_test).score(feature_test, response_test))

Forest fitted and response Vector predicted with one random_state in 3.2.1.:
Train set accuracy (CV=5):     0.7075093488418007
Validation set scores (CV=5):  0.5754446431676479
Test set accuracy:             0.7915492957746478 

Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
Train set accuracy (CV=5):     0.6974608304709671
Validation set scores (CV=5):  0.5825136001192339
Test set accuracy:             0.8070422535211268


### 3.3.5. Leave-one-out Cross-Validation

In [129]:
# Import necessary function
from sklearn.model_selection import LeaveOneOut

# Create objects
loocv = LeaveOneOut()

# Calculate & print scores
scores_o = cross_val_score(forest_o, feature_train, response_train)
scores_m = cross_val_score(forest_m, feature_train, response_train)

# Print results
print('Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print('LOOCV accuracy on train set:', np.mean(scores_o),'\n')
print('Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print('LOOCV accuracy on train set:', np.mean(scores_m))

Forest fitted and response Vector predicted with one random_state in 3.2.1.:
LOOCV accuracy on train set: 0.5793363684836553 

Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
LOOCV accuracy on train set: 0.5895596794821601


###   
###   
## 3.4. Precision Metrics

### 3.4.1. Confusion Matrix

In [130]:
# Import necessary function
from sklearn import metrics

# Make and print Confusion Matrix
confm_o = metrics.confusion_matrix(response_test, prediction_o)
print('Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(confm_o,'\n')

print('Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
confm_m = metrics.confusion_matrix(response_test, prediction_m.resulting_prediction)
print(confm_m)


# Define Variables
TP_o = confm_o[1, 1]  # True positives
TN_o = confm_o[0, 0]  # True negatives
FP_o = confm_o[0, 1]  # False positives
FN_o = confm_o[1, 0]  # False negatives

TP_m = confm_m[1, 1]  # True positives
TN_m = confm_m[0, 0]  # True negatives
FP_m = confm_m[0, 1]  # False positives
FN_m = confm_m[1, 0]  # False negatives


Forest fitted and response Vector predicted with one random_state in 3.2.1.:
[[ 76 229]
 [ 57 348]] 

Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
[[ 66 239]
 [ 51 354]]


### 3.4.2. Score

# THE LAST VERSION GIVES A DIFFERENT NUMBER BUT SHOULD BE THE SAME (SEE LDA ML)

In [131]:
print('Score for Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print((TP_o + TN_o) / (TP_o + TN_o + FP_o + FN_o))
print(metrics.accuracy_score(response_test,  prediction_o))
print(forest_o.score(feature_test, response_test),'\n')
print('Score for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print((TP_m + TN_m) / (TP_m + TN_m + FP_m + FN_m))
print(metrics.accuracy_score(response_test,  prediction_m.resulting_prediction))
print(forest_m.score(feature_test, response_test))

Score for Forest fitted and response Vector predicted with one random_state in 3.2.1.:
0.5971830985915493
0.5971830985915493
0.7915492957746478 

Score for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
0.5915492957746479
0.5915492957746479
0.8070422535211268


### 3.4.3. Error Rate

In [132]:
print('Error Rate for Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print((FP_o + FN_o) / (TP_o + TN_o + FP_o + FN_o))
print(1 - metrics.accuracy_score(response_test, prediction_o))
print(1 - forest_o.score(feature_test,response_test),'\n')

print('Error Rate for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print((FP_m + FN_m) / (TP_m + TN_m + FP_m + FN_m))
print(1 - metrics.accuracy_score(response_test, prediction_m.resulting_prediction))
print(1 - forest_m.score(feature_test,response_test))


Error Rate for Forest fitted and response Vector predicted with one random_state in 3.2.1.:
0.4028169014084507
0.4028169014084507
0.20845070422535217 

Error Rate for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
0.4084507042253521
0.4084507042253521
0.1929577464788732


### 3.4.4. Specifity

Fraction of correctly predicted negatives ('price down')

In [133]:
print('Specifity for Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(TN_o / (TN_o + FP_o),'\n')

print('Specifity for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print(TN_m / (TN_m + FP_m))

Specifity for Forest fitted and response Vector predicted with one random_state in 3.2.1.:
0.24918032786885247 

Specifity for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
0.21639344262295082


### 3.4.5. Sensitivity / Recall

Fraction of correctly predicted 'positives' ('price up')
###### True Positive Rate

In [134]:
print('Recall for Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(TP_o / (TP_o + FN_o))
print(metrics.recall_score(response_test, prediction_o),'\n')

print('Recall for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print(TP_m / (TP_m + FN_m))
print(metrics.recall_score(response_test, prediction_m.resulting_prediction))

Recall for Forest fitted and response Vector predicted with one random_state in 3.2.1.:
0.8592592592592593
0.8592592592592593 

Recall for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
0.8740740740740741
0.8740740740740741


### 3.4.5. Precision

How precise a positive (up) is predicted.

In [135]:
print('Precision for Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(TP_o / (TP_o + FP_o))
print(metrics.precision_score(response_test, prediction_o),'\n')

print('Precision for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print(TP_m / (TP_m + FP_m))
print(metrics.precision_score(response_test, prediction_m.resulting_prediction))

Precision for Forest fitted and response Vector predicted with one random_state in 3.2.1.:
0.6031195840554593
0.6031195840554593 

Precision for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
0.596964586846543
0.596964586846543


### 3.4.6. F-Score

In [136]:
print('F-Score for Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(metrics.f1_score(response_test, prediction_o))
print(((1+1**2) * TP_o)/((1+1**2) * TP_o + FN_o + FP_o),'\n')

print('Classification Report for this Forest:')
print(metrics.classification_report(response_test, prediction_o),'\n')

print('F-Score for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print(metrics.f1_score(response_test, prediction_m.resulting_prediction))
print(((1+1**2) * TP_m)/((1+1**2) * TP_m + FN_m + FP_m),'\n')

print('Classification Report for this Forest:')
print(metrics.classification_report(response_test, prediction_m.resulting_prediction))

F-Score for Forest fitted and response Vector predicted with one random_state in 3.2.1.:
0.7087576374745417
0.7087576374745418 

Classification Report for this Forest:
             precision    recall  f1-score   support

          0       0.57      0.25      0.35       305
          1       0.60      0.86      0.71       405

avg / total       0.59      0.60      0.55       710
 

F-Score for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
0.7094188376753507
0.7094188376753507 

Classification Report for this Forest:
             precision    recall  f1-score   support

          0       0.56      0.22      0.31       305
          1       0.60      0.87      0.71       405

avg / total       0.58      0.59      0.54       710

